In [ ]:
import os 
from collections import defaultdict
import numpy as np
import random
random.seed(4680)

num_evaluated = 200
topk = 200

run_dir = "/home/jupyter/unity_jointly_rec_and_search/experiments/unified_kgc/phase_1/experiment_10-02_005604/runs/"
rel_to_run_paths = {
    "is_similar_to": os.path.join(run_dir, "checkpoint_latest.test.sim.small.run"),
    "is_complementary_to": os.path.join(run_dir, "checkpoint_latest.test.compl.run"),
    "is_relevant_to": os.path.join(run_dir, "checkpoint_latest.test.query.small.run")
}
rel_to_qrels_path = {
    "is_similar_to": "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.sim.tsv",
    "is_complementary_to": "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.compl.tsv",
    "is_relevant_to": "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/qrels.test.tsv"
}
rel_to_qrels = {}
for rel, qrels_path in rel_to_qrels_path.items():
    qid_to_relpids = defaultdict(set)
    with open(qrels_path) as fin:
        for line in fin:
            qid, _, pid, _ = line.strip().split("\t")
            qid, pid = int(qid), int(pid)
            qid_to_relpids[qid].add(pid)
    
    rel_to_qrels[rel] = qid_to_relpids

rel_to_qid_to_pids = {}
rel_to_qid_to_labels = {}
for rel, run_path in rel_to_run_paths.items():
    qid_to_pids = defaultdict(list)
    qid_to_labels = defaultdict(list)
    qid_to_relpids = rel_to_qrels[rel]
    with open(run_path) as fin:
        for line in fin:
            qid, pid, _, _ = line.strip().split("\t")
            qid, pid = int(qid), int(pid)
            if qid not in qid_to_relpids:
                continue
            
            if len(qid_to_relpids[qid]) >= topk // 2:
                continue
            
            if len(qid_to_pids[qid]) == 0:
                qid_to_pids[qid] += list(qid_to_relpids[qid])
                qid_to_labels[qid] += [1.] * len(qid_to_relpids[qid])
            if len(qid_to_pids[qid]) < topk:
                if pid not in qid_to_relpids[qid]:
                    qid_to_pids[qid].append(pid)
                    qid_to_labels[qid].append(0.)
    
    assert np.mean([len(v) for v in qid_to_pids.values()]) - 200. < 1e-4, np.mean([len(v) for v in qid_to_pids.values()])
    rel_to_qid_to_pids[rel] = qid_to_pids
    rel_to_qid_to_labels[rel] = qid_to_labels

out_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_rerank_evaluate/"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

rel_to_out_path = {
    "is_similar_to": os.path.join(out_dir, "pairs.sim.rerank"),
    "is_complementary_to": os.path.join(out_dir, "paris.compl.rerank"),
    "is_relevant_to": os.path.join(out_dir, "pairs.query.rerank")
}
for rel, out_path in rel_to_out_path.items():
    qid_to_pids = rel_to_qid_to_pids[rel]
    qid_to_labels = rel_to_qid_to_labels[rel]
    candidate_qids = list(qid_to_pids.keys())
    random.shuffle(candidate_qids)
    candidate_qids = candidate_qids[:num_evaluated]
    
    with open(out_path, "w") as fout:
        for qid in candidate_qids:
            for pid, label in zip(qid_to_pids[qid], qid_to_labels[qid]):
                fout.write(f"{qid}\t{pid}\t{label}\t{rel}\n")
                
    ! wc -l $out_path
    ! head -n 3 $out_path
    ! tail -n 3 $out_path
    print("="*50)
            

In [ ]:
path_1 = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_rerank_evaluate/pairs.sim.rerank"
path_2 = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_rerank_evaluate/paris.compl.rerank"
path_3 = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_rerank_evaluate/pairs.query.rerank"
path = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_rerank_evaluate/pairs.all.rerank"

! cat $path_1 $path_2 $path_3 >> $path
! wc -l $path
! head -n 3 $path
! tail -n 3 $path